In [1]:

# External Dependencies
import os                          # Access system file-tree
import sys                         # Modify system parameters
from math import isclose           # isclose()
import numpy as np                 # Data structures
from optparse import OptionParser  # Get user input
import matplotlib.ticker as mticker  # Scientific notation in labels
from matplotlib.ticker import FuncFormatter

np.set_printoptions(threshold=sys.maxsize, precision=16)

In [2]:
# Internal Dependencies
internalPath = os.getcwd() + "/../python"
print(internalPath)
sys.path.insert(0, internalPath)
from plotStyling import PlotStyling  # noqa: E402
from GSDUtil import GSDUtil  # noqa: E402

/home/aglisman/Dropbox/3_Brady_Lab/Code/potential-flow/bodies-in-potential-flow/jupyter_notebooks/../python


In [3]:
# Parameters
relative_path_base = os.getcwd() + "/../data_tag/v1.0.0_output_last_frame"
relative_path = relative_path_base + "/2021-08-27.09-57-48_collinear-swimmer_varyRelDisp"
output_dir = "figures"
output_dir = relative_path + "/" + output_dir + "/"
gsd_files = []
epsOutput = True


# Correctly get scientific notation in text elements
def scientific(x, pos):
    return '%0.2e' % x


scientific_formatter = FuncFormatter(scientific)
fmt = mticker.FuncFormatter(scientific_formatter)

In [15]:
# Decompress data

os.system(f"\\cat {relative_path}.tar.xz.parta* > {relative_path}.tar.xz")
os.system(f"tar xvf {relative_path}.tar.xz -C {relative_path_base}")

0

In [16]:
# Load data from GSD

try:
    # Loop through all subdirectories in the 'data' directory
    for root, dirs, files in os.walk(relative_path, topdown=True):
        dirs.sort()
        files.sort()

        for file in files:  # Loop through all files in given directory
            if (".gsd" in file):
                with open(root + "/" + file) as g:
                    cur_gsd = GSDUtil(g.name, create_gsd=False)
                    gsd_files.append(cur_gsd)

    assert(len(gsd_files) > 0)

except:  # No files found
    raise IOError(
        f"Failure to load data. No files found in relPath {relative_path}")

In [17]:
# Remove decompressed data

os.system(f"rm -rf {relative_path}.tar.xz")
os.system(f"rm -rf {relative_path}")

0

In [18]:
# Load data into np structures

CoM_disp = np.zeros(len(gsd_files), dtype=np.double)
R_avg = np.zeros_like(CoM_disp, dtype=np.double)
phaseShift = np.zeros_like(CoM_disp, dtype=np.double)
U0 = np.zeros_like(CoM_disp, dtype=np.double)
omega = np.zeros_like(CoM_disp, dtype=np.double)
epsilon = np.zeros_like(CoM_disp, dtype=np.double)
final_t = np.zeros_like(CoM_disp, dtype=np.double)

for i in range(len(gsd_files)):

    # Data from final frame
    gsd_files[i].snapshot = gsd_files[i].trajectory.read_frame(
        gsd_files[i].trajectory.file.nframes - 1)
    final_t[i] = gsd_files[i].snapshot.log['integrator/t']
    CoM_disp_comp = gsd_files[i].snapshot.log['particles/double_position'][1]

    # Data from initial frame (not 0)
    gsd_files[i].snapshot = gsd_files[i].trajectory.read_frame(1)
    R_avg[i] = gsd_files[i].snapshot.log['swimmer/R_avg']
    phaseShift[i] = gsd_files[i].snapshot.log['swimmer/phase_shift']
    U0[i] = gsd_files[i].snapshot.log['swimmer/U0']
    omega[i] = gsd_files[i].snapshot.log['swimmer/omega']
    epsilon[i] = U0[i] / R_avg[i] / omega[i]
    CoM_disp_comp -= gsd_files[i].snapshot.log['particles/double_position'][1]

    CoM_disp[i] = np.linalg.norm(CoM_disp_comp)

In [19]:
# NOTE: Assuming each GSD has same number of frames
nframes = gsd_files[0].trajectory.file.nframes

# ANCHOR: Orientation vector (not unit norm)
q = np.zeros((len(gsd_files), 3, nframes - 2), dtype=np.double)
q0 = np.zeros((len(gsd_files), 3), dtype=np.double)

# ANCHOR: temporal data
time = np.zeros((len(gsd_files), nframes - 2), dtype=np.double)

# ANCHOR: kinematic data (simulation_number, particle_number, dimension_number, frame_number)
positions = np.zeros((len(gsd_files), 3, 3, nframes - 2), dtype=np.double)
velocities = np.zeros_like(positions, dtype=np.double)
accelerations = np.zeros_like(positions, dtype=np.double)

In [20]:
for i in range(len(gsd_files)):  # loop over all GSD files

    gsd_current = gsd_files[i]
    q_current = np.zeros((3, nframes - 2), dtype=np.double)
    q_current_norm = np.zeros((nframes - 2), dtype=np.double)

    for j in range(1, nframes - 1):  # loop over all frames in GSD file

        snapshot_current = gsd_current.trajectory.read_frame(j)

        q_current[:, j - 1] = snapshot_current.log['particles/double_position'][0]
        q_current[:, j - 1] -= snapshot_current.log['particles/double_position'][2]

        time[i, j - 1] = snapshot_current.log['integrator/t']

        positions[i, :, :, j -
                  1] = snapshot_current.log['particles/double_position']
        velocities[i, :, :, j -
                   1] = snapshot_current.log['particles/double_velocity']
        accelerations[i, :, :,  -
                      1] = snapshot_current.log['particles/double_moment_inertia']

    q[i, :, :] = q_current

q0 = q[:, :, 0]

In [21]:
q0 = q[:, :, 0]
q0_norm = np.linalg.norm(q0, axis=1)
q0 = np.divide(q0, q0_norm[:, np.newaxis])

print(q0)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [23]:
# ANCHOR: Angular orientational displacement
theta = np.zeros((len(gsd_files), nframes - 2), dtype=np.double)
theta_norm = np.zeros((nframes - 2), dtype=np.double)

for i in range(len(gsd_files)):

    theta[i, :] = np.dot(q0[i, :].reshape(1, 3), q[i, :, :])

    print(np.array(theta[i, 0] - theta[i, -1], dtype=np.double))

    theta_norm = np.linalg.norm(q0[i, :]) * np.linalg.norm(q[i, :, :], axis=0)
    theta[i, :] = theta[i, :] / theta_norm

print(np.array(theta[:, 0] - theta[:, -1], dtype=np.double))

2.1014123774421023e-10
2.1043167208745217e-10
2.1018387030835584e-10
2.1030466257343505e-10
2.1049473275525088e-10
2.1011992146213743e-10
2.1051960175100248e-10
2.100195573007113e-10
2.1033397246128516e-10
2.1027446450716525e-10
2.1038637498804746e-10
2.1025314822509245e-10
2.1020873930410744e-10
2.1013857320895113e-10
2.1036239417071556e-10
2.1025758911719095e-10
2.1042634301693397e-10
2.1009860518006462e-10
2.1047341647317808e-10
2.1032420249866846e-10
2.1005508443749932e-10
2.1028601082662135e-10
2.1033397246128516e-10
2.1031887342815025e-10
2.106919083644243e-10
2.1021406837462564e-10
2.1006485440011602e-10
2.1030999164395325e-10
2.1015367224208603e-10
2.099511675623944e-10
2.1088908397359774e-10
2.1080737155898532e-10
2.1002577454964921e-10
2.1035173602967916e-10
2.1049118004157208e-10
2.1030999164395325e-10
2.099653784171096e-10
2.107274355012123e-10
2.098730078614608e-10
2.1021406837462564e-10
2.0950707835254434e-10
2.099653784171096e-10
2.0980905901524238e-10
2.1090329482831294

In [24]:
# relative separation between particle pairs
R_loc = positions[:, 1, :, :]
R_loc_init = positions[:, 1, :, 0]
DR_loc = R_loc - R_loc_init[:, :, np.newaxis]
R_12 = positions[:, 0, :, :] - R_loc
R_32 = positions[:, 2, :, :] - R_loc
# distance between particle pairs
Dr_Loc = np.linalg.norm(DR_loc, axis=1)
r_12 = np.linalg.norm(R_12, axis=1)
r_32 = np.linalg.norm(R_32, axis=1)

print(r_12[0, 0]-r_12[0, -1])
print(time[0, -1] - 1.0)

1.736761845450019e-10
7.918110611626616e-12


In [25]:
# relative velocities between particle pairs
U_loc = velocities[:, 1, :, :]
U_12 = velocities[:, 0, :, :] - U_loc
U_32 = velocities[:, 2, :, :] - U_loc
# relative velocity norms between particle pairs
u_12 = np.linalg.norm(U_12, axis=0)
u_32 = np.linalg.norm(U_32, axis=0)

In [26]:
 # Average locater point velocity
U_loc_mean = np.mean(U_loc, axis=2)
U_loc_mean_x = U_loc_mean[:, 0]

print(U_loc_mean.shape)
print(U_loc_mean_x)

(53, 3)
[ 9.8093719130640744e-02  8.5215153660758172e-02  8.1440483077825740e-02
  7.7928232986463258e-02  5.2836094505237095e-02  4.4670414359441170e-02
  3.8247900213850716e-02  3.3089382669166417e-02  2.8875429750148559e-02
  2.5384690022200455e-02  2.2458878812655500e-02  9.8044402736371883e-02
  1.9981824771323948e-02  1.6045923997949574e-02  1.3094015048722367e-02
  1.0828339142636768e-02  9.0561795773012165e-03  7.6479188081020810e-03
  6.5136151297305340e-03  5.5892203658258676e-03  4.8281160562057366e-03
  4.1957270335918253e-03  9.7995127516833216e-02  3.6659882889393761e-03
  3.2189665405364205e-03  2.8392226208148520e-03  2.5146614107183834e-03
  2.2357099019000254e-03  1.9947205179305398e-03  1.5206862527859715e-03
  1.1793154576197737e-03  9.2788926966611549e-04  7.3909140458565446e-04
  9.7945893420623598e-02  5.9491122170296875e-04  4.8315724742055389e-04
  3.9538609895990203e-04  3.2563135718675354e-04  1.5630371954474545e-04
  7.6324091344670214e-05  3.449972756897992

In [27]:
# Sort the average locater velocities by R_avg
idx = np.argsort(R_avg)

U_loc_mean_x_srt = U_loc_mean_x[idx]
R_avg_srt = R_avg[idx]

In [28]:
# Get average velocity when R_avg = 4.0 (what was done in internal dynamics plot)
U_loc_mean_x_desired = U_loc_mean_x_srt[R_avg_srt == 4.0]
print(f"Average velocity over period of articulation: {U_loc_mean_x_desired}")

Average velocity over period of articulation: [0.0019947205179305]
